# API

In [0]:
!pip install kafka-python

     |████████████████████████████████| 246 kB 5.2 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
!pip install geopy

     |████████████████████████████████| 118 kB 6.0 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


Ahora se proseguirá a hacer la conexión. Primero se crea la función `data(Diccionario)`, que recibe el diccionario con los datos generado por la API, se extraen las características deseadas: id, latitud, longitud (del avión en su posición actual), su estatus (esxplicado posteriormente), y el aeropuerto del que proviene.

In [0]:
# Definimos una función que estructure los datos del Json

def data(Diccionario):
    dat = {}
    dat['id'] =  Diccionario['callsign']
    dat['lat'] = str(Diccionario['positions'][0]['lat'])
    dat['lon'] = str(Diccionario['positions'][0]['lon'])
    #dat['time'] = Diccionario['positions'][0]['date']
    dat['status'] = status((dat['lat'], dat['lon']))
    dat['from'] = Diccionario['departureAirportFsCode']
    return ",".join(list(dat.values()))

A continuación se puede observar el mapa de las rutas aéreas de Aeropuerto Internacional de la Ciudad de México (AICM).

![Image of Yaktocat](https://images.adsttc.com/media/images/606e/225f/f91c/81ab/3800/00f6/slideshow/PORTADA.jpg?1617830489)

Por lo anterior se puede hacer la suposición de que los vuelos que estén a 20 km del AICM con el destino a la Ciudad de México, están próximos a aterrizar o ya están aterrizando (por el retraso de los datos que tiene el API). Así que la función `status(coord_2)`, recive unas coordenadas y si está a menos de 20 km el estado que devuelve es "L" (Landing), de lo contrario retorna "A" (Active).

In [0]:
# Generamos la función que especifique el estatus del vuelo.

def status(coords_2):
    coords_1 = (19.436111, -99.071944)
    dis =  geopy.distance.geodesic(coords_1, coords_2).km
    if dis < 20:
        return('L')
    else:
        return('A')

Después se define la función `Vuelos_Tracker(air_code, maxflights)` que hace consultas del API, dado un código de aeropuerto y un número máximo de respeuesta devuelve los vuelos con destino a dicho aeropuerto más cercanos. La sonsulta se hace a través de un request a la URL con la estructura definida en `url_req`, se devuelven los datos en formato JSON.

In [0]:
import requests
from kafka import KafkaProducer
import json
import geopy.distance

# Definimos una función que geneere la consulta a partir de la api
def Vuelos_Tracker(air_code, maxflights):
    url = 'https://api.flightstats.com/flex/flightstatus/rest/v2/json/airport/tracks/'

    appkey = '5fddd1d9206efb60b24ff5b987a26d47'
    appid = 'c86b7d77'
    maxflights = str(maxflights)
    url_req = url + air_code + '/arr?appId=' + appid + '&appKey=' + appkey + '%09&includeFlightPlan=false&maxPositions=1&maxFlights=' + maxflights
    
    response = requests.request("GET", url_req).json()
    return response

# Generamoos una prueba
prueba = Vuelos_Tracker('MEX', 100)
prueba

Out[5]: {'request': {'includeFlightPlan': {'requested': 'false', 'interpreted': False},
  'maxFlights': {'requested': '100', 'interpreted': 100},
  'maxPositions': {'requested': '1', 'interpreted': 1},
  'url': 'https://api.flightstats.com/flex/flightstatus/rest/v2/json/airport/tracks/MEX/arr?includeFlightPlan=false&maxFlights=100&maxPositions=1',
  'airport': {'requestedCode': 'MEX', 'fsCode': 'MEX'}},
 'appendix': {'airlines': [{'fs': 'AA',
    'iata': 'AA',
    'icao': 'AAL',
    'name': 'American Airlines',
    'active': True},
   {'fs': 'VW',
    'iata': 'VW',
    'icao': 'TAO',
    'name': 'Aeromar',
    'active': True},
   {'fs': 'KL', 'iata': 'KL', 'icao': 'KLM', 'name': 'KLM', 'active': True},
   {'fs': 'AC',
    'iata': 'AC',
    'icao': 'ACA',
    'name': 'Air Canada',
    'active': True},
   {'fs': 'VVC',
    'iata': 'VH',
    'icao': 'VVC',
    'name': 'Viva Air Colombia',
    'active': True},
   {'fs': 'LP',
    'iata': 'LP',
    'icao': 'LPE',
    'name': 'LATAM Airlines

Como los datos que ofrece la API no son streaming, lo simularemos, para ello primero se inicia el productor de la siguente manera:

In [0]:
# Iniciamos el productor.
producer = KafkaProducer(bootstrap_servers='localhost:9092')

Y finalmente se simula el flujo de datos con un `while` que dura 500 segundos en total, cada 30 segundo se hace una consulta y se manda la información que produce la función `data()`, previamente explicada.

In [0]:
# Iniciamos la simulación del flujo

import time

t = time.time()
while True:
    prueba = Vuelos_Tracker('MEX', 100)
    for ubi in prueba['flightTracks']:
        coor = data(ubi)
        print(coor)
        producer.send("proyecto", coor.encode('utf-8'))
    time.sleep(30)
    if (time.time() - t) > 2000:
       break